In [54]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# 載入 Iris 資料集
iris = load_iris()
X = iris.data  # 特徵: sepal length, sepal width, petal length, petal width
y = iris.target  # 類別標籤

# 資料標準化
scaler = StandardScaler()
X = scaler.fit_transform(X)

# 分類任務: 將 y one-hot 編碼
encoder = OneHotEncoder(sparse_output=False)
y_classification = encoder.fit_transform(y.reshape(-1, 1))

# 回歸任務: 使用原始類別數字 (不做 One-hot)
y_regression = y.astype(np.float32)

# 分割訓練和測試資料
X_train, X_test, y_train_cls, y_test_cls = train_test_split(X, y_classification, test_size=0.2, random_state=42)
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X, y_regression, test_size=0.2, random_state=42)

# 轉為 PyTorch Tensor
X_train, X_test = torch.tensor(X_train, dtype=torch.float32), torch.tensor(X_test, dtype=torch.float32)
y_train_cls, y_test_cls = torch.tensor(y_train_cls, dtype=torch.float32), torch.tensor(y_test_cls, dtype=torch.float32)
X_train_reg, X_test_reg = torch.tensor(X_train_reg, dtype=torch.float32), torch.tensor(X_test_reg, dtype=torch.float32)
y_train_reg, y_test_reg = torch.tensor(y_train_reg, dtype=torch.float32), torch.tensor(y_test_reg, dtype=torch.float32)


In [ ]:
class ClassificationModel(nn.Module):
    def __init__(self):
        super(ClassificationModel, self).__init__()
        self.fc1 = nn.Linear(4, 16)  # 輸入層到隱藏層
        self.fc2 = nn.Linear(16, 8)  # 隱藏層到隱藏層
        self.fc3 = nn.Linear(8, 3)   # 隱藏層到輸出層
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return self.softmax(x)

# 初始化模型
model_cls = ClassificationModel()
criterion_cls = nn.CrossEntropyLoss()
optimizer_cls = optim.Adam(model_cls.parameters(), lr=0.01)

# 訓練模型
for epoch in range(100):
    optimizer_cls.zero_grad()
    outputs = model_cls(X_train)
    loss = criterion_cls(outputs, torch.argmax(y_train_cls, dim=1))
    loss.backward()
    optimizer_cls.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/100], Loss: {loss.item():.4f}')


Epoch [10/100], Loss: 1.0334
Epoch [20/100], Loss: 0.8819
Epoch [30/100], Loss: 0.7345
Epoch [40/100], Loss: 0.6658
Epoch [50/100], Loss: 0.6183
Epoch [60/100], Loss: 0.5943
Epoch [70/100], Loss: 0.5844
Epoch [80/100], Loss: 0.5791
Epoch [90/100], Loss: 0.5761
Epoch [100/100], Loss: 0.5742


In [58]:
from sklearn.metrics import accuracy_score

# 模型評估
model_cls.eval()  # 設置模型為評估模式
with torch.no_grad():
    outputs = model_cls(X_test)  # 預測輸出
    predicted = torch.argmax(outputs, dim=1)  # 獲取預測的類別
    actual = torch.argmax(y_test_cls, dim=1)  # 真實的類別

accuracy = accuracy_score(actual.cpu(), predicted.cpu())  # 計算準確率
print(f'Accuracy: {accuracy:.4f}')


Accuracy: 1.0000


In [61]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(actual.cpu(), predicted.cpu(), average='weighted')  # 加權平均精確率
recall = recall_score(actual.cpu(), predicted.cpu(), average='weighted')        # 加權平均召回率
f1 = f1_score(actual.cpu(), predicted.cpu(), average='weighted')                # 加權平均 F1-score

print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')


from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(actual.cpu(), predicted.cpu())
print('Confusion Matrix:')
print(conf_matrix)


Precision: 1.0000
Recall: 1.0000
F1 Score: 1.0000
Confusion Matrix:
[[10  0  0]
 [ 0  9  0]
 [ 0  0 11]]


In [56]:
class RegressionModel(nn.Module):
    def __init__(self):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(4, 16)  # 輸入層到隱藏層
        self.fc2 = nn.Linear(16, 8)  # 隱藏層到隱藏層
        self.fc3 = nn.Linear(8, 1)   # 隱藏層到輸出層

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# 初始化模型
model_reg = RegressionModel()
criterion_reg = nn.MSELoss()
optimizer_reg = optim.Adam(model_reg.parameters(), lr=0.001)

# 訓練模型
for epoch in range(200):
    optimizer_reg.zero_grad()
    outputs = model_reg(X_train_reg)
    loss = criterion_reg(outputs.flatten(), y_train_reg)
    loss.backward()
    optimizer_reg.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/100], Loss: {loss.item():.4f}')


Epoch [10/100], Loss: 2.0493
Epoch [20/100], Loss: 1.8766
Epoch [30/100], Loss: 1.7096
Epoch [40/100], Loss: 1.5446
Epoch [50/100], Loss: 1.3740
Epoch [60/100], Loss: 1.1951
Epoch [70/100], Loss: 1.0097
Epoch [80/100], Loss: 0.8241
Epoch [90/100], Loss: 0.6469
Epoch [100/100], Loss: 0.4879
Epoch [110/100], Loss: 0.3567
Epoch [120/100], Loss: 0.2602
Epoch [130/100], Loss: 0.1978
Epoch [140/100], Loss: 0.1628
Epoch [150/100], Loss: 0.1444
Epoch [160/100], Loss: 0.1330
Epoch [170/100], Loss: 0.1237
Epoch [180/100], Loss: 0.1152
Epoch [190/100], Loss: 0.1074
Epoch [200/100], Loss: 0.1005


In [53]:
from sklearn.metrics import mean_squared_error, r2_score

# 預測測試資料
with torch.no_grad():
    y_pred_reg = model_reg(X_test_reg).flatten()  # 預測值
    y_true_reg = torch.round(y_test_reg).int()  # 真實值 to round integer

# 計算 RMSE 和 R²
rmse_reg = mean_squared_error(y_true_reg.numpy(), y_pred_reg.numpy())
r2_reg = r2_score(y_true_reg.numpy(), y_pred_reg.numpy())

print(f'Regression Model RMSE: {rmse_reg:.4f}')
print(f'Regression Model R²: {r2_reg:.4f}')


Regression Model RMSE: 0.0945
Regression Model R²: 0.8647
